In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [2]:
empl_roll = pd.DataFrame(pd.read_csv('CR Indexed Worker Roster - Sales 2021-01-04 19_16 CST.xlsx - Sheet1.csv'))
user_df = pd.DataFrame(pd.read_csv('ihm_users.csv'))
market_owners = pd.DataFrame(pd.read_csv('ihm_account_management.csv'))
workday = pd.DataFrame(pd.read_csv('usersChanges_2021_02_15_11_54.csv'))
revenue = pd.DataFrame(pd.read_csv('duplicates.csv'))
projects = pd.DataFrame(pd.read_csv('iheart_projects.csv'))

ParserError: Error tokenizing data. C error: Expected 12 fields in line 1989, saw 13


In [ ]:
# empl_roll[empl_roll.apply(lambda row: row.astype(str).str.contains('Albany', case=False).any(), axis=1)].head()

In [ ]:
# revenue[revenue.apply(lambda row: row.astype(str).str.contains('Hatties', case=False).any(), axis=1)].head()

# Task:

### Read-in Employee Roll
### Read-in Usership Database Pull
### Read-in Monday.com market ownership report
### Create lists of markets by owner
### Append owners name in new "Owner" column in user_df
### Sort by first project date OR date of hire (tentative pending LW)

In [ ]:
# # Creates a list for each team member of the markets he/she owns
# Casts group names to lowercase to aid in matching
user_df.group = user_df['group'].str.lower()
# Casts market names to lowercase to aid in matching
market_owners.Name = market_owners.Name.str.lower()

arliss = market_owners[market_owners['Owner'] == 'Arliss Coates']
arliss = arliss.Name.tolist()

ashley = market_owners[market_owners['Owner'] == 'Ashley McCarthy']
ashley = ashley['Name'].tolist()

jenny = market_owners[market_owners['Owner'] == 'Jenny Donheiser']
jenny = jenny['Name'].tolist()

mariana = market_owners[market_owners['Owner'] == 'Mariana E Trevino Stafford']
mariana = mariana['Name'].tolist()

# Strips whitespace from group names in user_df
user_df.group = user_df.group.str.strip()

# # renaming special cases due to human error
user_df.loc[(user_df.group == 'mccallen, tx'),'group'] = 'mcallen, tx'
user_df.loc[(user_df.group == 'modesto / stockton, ca'), 'group'] = 'modesto/stockton, ca'
user_df.loc[(user_df.group == 'st. petersburg, fl | inside sales'), 'group'] = 'inside sales'
user_df.loc[(user_df.group == 'tuscaloose, al'), 'group'] = 'tuscaloosa, al'
# Lexington and Somerset are distinct markets in iHeart's system
user_df.loc[(user_df.group == 'lexington / somerset, ky'), 'group'] = 'lexington, ky'
user_df.loc[(user_df.group == 'roanoke / lynchburg, va'), 'group'] = 'roanoke/lynchburg, va'

# # Slicing user_df by market owner

arliss_users = user_df.loc[user_df['group'].isin(arliss)]

ashley_users = user_df.loc[user_df['group'].isin(ashley)]

jenny_users = user_df.loc[user_df['group'].isin(jenny)]

mariana_users = user_df.loc[user_df['group'].isin(mariana)]

print('arliss_users: ' + str(arliss_users.shape), '\n'
'ashley_users: ' + str(ashley_users.shape), '\n'
'jenny_users: ' + str(jenny_users.shape), '\n'
'mariana_users: ' + str(mariana_users.shape))

print('arliss projects/users: ' + str(arliss_users.projects.sum()/arliss_users.shape[0]), '\n'
      'arliss projects: ' + str(arliss_users.projects.sum()), '\n'
      'ashley projects/users: ' + str(ashley_users.projects.sum()/arliss_users.shape[0]), '\n'
      'ashley projects: ' + str(ashley_users.projects.sum()), '\n'
      'jenny projects/users: ' + str(jenny_users.projects.sum()/arliss_users.shape[0]), '\n'
      'jenny projects: ' + str(jenny_users.projects.sum()), '\n'
      'mariana projects/users: ' + str(mariana_users.projects.sum()/arliss_users.shape[0]), '\n'
      'mariana projects: ' + str(mariana_users.projects.sum()))


# # Sense checking; making sure the number of markets in each team member's subset of df_users
# # matches the full list of that team member's markets

print('arliss groups in user_df: ' + str(len(arliss_users.group.unique())), '\n'
      'arliss groups in market_owners :' + str(len(arliss)))

print('ashley groups in user_df: ' + str(len(ashley_users.group.unique())), '\n'
      'ashley groups in market_owners: ' + str(len(ashley)))

print('jenny groups in user_df: ' + str(len(jenny_users.group.unique())), '\n'
      'jenny groups in market_owners: ', len(jenny))

print('mariana groups in user_df: ', len(mariana_users.group.unique()), '\n'
      'mariana groups in market_owners: ', len(mariana))

print('users_df shape: ' + str(user_df.shape))
print('empl_roll shape: ' + str(empl_roll.shape))

In [3]:
# # Adding new field "owner" to each team member's subset of user_df for the purpose of concatenating
# # these dataframes in the following code
arliss_users['owner'] = 'Arliss'
ashley_users['owner'] = 'Ashley'
jenny_users['owner'] = 'Jenny'
mariana_users['owner'] = 'Mariana'

NameError: name 'arliss_users' is not defined

#### Defines variable "frames" as a list of the user_df subsets so as
#### to concatenate them in the following line. Redefining variable "user_df"
#### as the same dataframe but with "owner" column added.

In [298]:
frames = [arliss_users, ashley_users, jenny_users, mariana_users]
user_df = pd.concat(frames)

user_df.to_csv('ihm_users_with_owner.csv', index = 0)

empl_roll['Email - Primary Work'] = empl_roll['Email - Primary Work'].str.lower()
empl_roll.rename(columns = {'Email - Primary Work': 'email'}, inplace=True)

user_df.email = user_df.email.str.lower()
filtered_users = user_df.groupby('email').filter(lambda x : len(x)==1)

empl_roll.Market = empl_roll.Market.str.lower()
empl_roll.Market = [x.replace(' - market', '') for x in empl_roll.Market]

merged = pd.DataFrame(pd.merge(user_df, empl_roll, on='email', how='outer'))

merged.to_csv('merged_hr_and_users.csv')



print('user_df shape is: ', user_df.shape, '\n'
     'empl_roll shape is: ', empl_roll.shape ,'\n'
     'registered users minus employees on file: ', user_df.shape[0] - empl_roll.shape[0], '\n'
     'merged shape is: ', merged.shape)

user_df shape is:  (3112, 9) 
empl_roll shape is:  (2183, 17) 
registered users minus employees on file:  929 
merged shape is:  (3499, 25)


In [303]:
merged.loc[merged['name'].str.split().str.len() == 2, 'first name'] = merged['name'].str.split().str[0]

virginia = merged.loc[(merged.projects <= 5) &
                     (merged['Region/Area'].str.contains('Virginia')) &
                     merged['Job Title'].str.contains('Acc')].groupby('email').filter(lambda x : len(x)== 1)[['email', 'first name', 'Market', 'projects']]
# virginia

In [304]:
# empl_roll[empl_roll['Job Title'].str.contains('Account', na=False)]['Job Title'].value_counts()

In [314]:
# empl_roll, user_df, workday
allmerge = pd.DataFrame(pd.merge(merged, workday,  on='email', how='outer'))

deletes = workday.loc[workday.action == 'DELETE'].action.tolist()


allmerge.to_csv('allmerge.csv')

In [348]:
projects.opportynity_id = projects.opportynity_id.astype(str)

projects.opportynity_id = [x[:15] for x in projects.opportynity_id]

projects.rename(columns = {'opportynity_id': 'opportunity_id'}).head()

project_merge = pd.DataFrame(pd.merge(revenue, projects, on='project_id', how='outer'))

projects_people_merge = pd.DataFrame(pd.merge(allmerge, project_merge, on='email', how='outer')).fillna('no_data', inplace = True)

projects_people_merge.rename(columns = {'client': 'adwire_client', 'group_x': 'adwire_group', 'name_x': 'adwire_name_x',
                                       'email': 'adwire_email', 'projects': 'adwire_projects', 'spend': 'adwire_spend',
                                       'project_ts': 'adwire_first_posting', 'group_cap': 'adwire_cap', 'owner': 'account_owner',
                                       'Worker': 'hr_worker', 'Job Title': 'hr_job_title', 'Business Title': 'hr_business_title',
                                       'Hire Date': 'hr_hire_date', 'Phone - Primary Work': 'hr_work_phone',
                                       'Mobile Phones': 'hr_mobile_phone', "Worker's Manager": 'hr_worker_manager',
                                        "Worker's Manager Business Title": "hr_worker_manager_business_title",
                                       "Worker's Manager's Manager": "hr_worker_manager_manager",
                                       "Worker's Manager's Manager Business Title": "hr_worker_manager_manager_business_title",
                                       'Market': 'hr_market', 'Market President': 'hr_market_president', 'Region/Area': 'hr_region/area',
                                       'Area/Region/Community President': 'hr_area/region/community_president', 'Division': 'hr_division',
                                       'Division President': 'hr_division_president', 'first name_x': 'workday_first_name', 
                                       'action': 'workday_action', 'client code': 'workday_client_code', 'first name_y': 'workday_first_name2',
                                       'last name': 'workday_last_name', 'group_y': 'workday_group_y', 'stage': 'revenue_stage',
                                        'market_division': 'revenue_market_division', 'market_area': 'revenue_market_area', 'market': 'revenue_market',
                                       'ae_name': 'revenue_ae_name', 'new_business': 'revenue_new_business', 'account': 'revenue_account',
                                       'opportunity': 'revenue_opportunity', 'project_id': 'revenue_project_id', 'form_date': 'revenue_form_date',
                                       'opportunity_created_date': 'revenue_opportunity_created_date', 'opportunity_closed_date': 'revenue_opportunity_closed_date',
                                       'gross_expected_revenue': 'revenue_gross_expected_revenue', 'gross_amount': 'revenue_gross_amount',
                                        'form_created_on_closed_opp': 'revenue_form_created_on_closed_opp',
                                       'market_type': 'revenue_market_type', 'create_ts': 'adwire_post_date', 'name_y': 'adwire_name_y',
                                       'group': 'adwire_group2', 'submissions': 'adwire_submissions', 'opportynity_id': 'adwire_opportunity_id',
                                       'credits_used': 'adwire_credits_used', 'industry': 'adwire_industry', 'tone': 'adwire_tone'}, inplace=True)

projects_people_merge.to_csv('fullmerge.csv')

In [330]:
projects_people_merge.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['client', 'group_x', 'name_x', 'email', 'projects', 'spend',
       'project_ts', 'group_cap', 'owner', 'Worker', 'Job Title',
       'Business Title', 'Hire Date', 'Phone - Primary Work', 'Mobile Phones',
       'Worker's Manager', 'Worker's Manager Business Title',
       'Worker's Manager's Manager',
       'Worker's Manager's Manager Business Title', 'Market',
       'Market President', 'Region/Area', 'Area/Region/Community President',
       'Division', 'Division President', 'first name_x', 'action',
       'client code', 'first name_y', 'last name', 'group_y', 'stage',
       'market_division', 'market_area', 'market', 'ae_name', 'new_business',
       'account', 'opportunity', 'project_id', 'form_date',
       'opportunity_created_date', 'opportunity_closed_date',
       'gross_expected_revenue', 'gross_amount', 'form_created_on_closed_opp',
       'market_type', 'create_ts', 'name_y', 'group', 'submissions',
       'opportynity_id', 